# Figures of Merit in `vipdopt`

This notebook serves as a guide to using the figure of merit (FoM) class `FoM`. This
class has a number of tools designed to abstract the notion of a FoM.

The FoMs in `vipdopt` are based on the adjoint state method.<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)

For later cells to work, please run the following block first.


<a name="cite_note-1"></a>1. [^](#cite_ref-1) Lalau-Keraly, C. M., Bhargava, S., Miller, O. D. & Yablonovitch, E. Adjoint shape optimization applied to electromagnetic design. Opt. Express 21, 21693 (2013).

In [ ]:
# imports
from pathlib import Path
import sys  

import numpy as np

np.set_printoptions(threshold=100)

# Get vipdopt directory path from Notebook
parent_dir = str(Path().resolve().parents[1])

# Add to sys.path
sys.path.insert(0, parent_dir)

# Imports from vipdopt
from vipdopt.optimization import FoM, BayerFilterFoM
from vipdopt.simulation import Power, Profile, LumericalSimulation, GaussianSource, DipoleSource, LumericalFDTD


## Using the `FoM` Class 

The `FoM` class at its core is defined by two functions:
* A function that computes the FoM (`fom_func`)
* A function that computes the gradient of the FoM for optimization (`grad_func`)

To instantiate a `FoM` object, these two functions will need to be provided as well as
a number of other arguments:
* Which polarization to use out of TE, TM, or TE + TM (`polarization`)
* A list of sources to enable in the forward / adjoint simulations (`fwd_srcs` / `adj_srcs`)
* A list of monitors in the forward / adjoint simulations from which data will be accessed in computing the FoM (`fwd_monitors` / `adj_monitors`)
* A list of the indices of frequencies being minimized / maximized in the optimization (`pos_max_freqs` / `neg_min_freqs`)

Note that monitors will output data of shape N x ... x L where L is the total number of
frequencies. So specifying that `pos_max_freqs=range(L)` is equivalent to saying that
we're maximizing over every frequency.

In the following example, we make use of the `BayerFilterFoM` subclass, which has
pre-defined functions for the FoM and gradient.

In [ ]:
# Note: all object names are based on those in "simulation_example.json"


# List of sources needed to make forward / adjoint simulations
forward_sources = [
    GaussianSource('forward_src_x'),
]
adjoint_sources = [
    GaussianSource('forward_src_x'),
    DipoleSource('adjoint_src_0x'),
]

# Monitors whose data is necessary for computing the FoM
forward_monitors = [
    Power('focal_monitor_0'),
    Power('transmission_monitor_0'),
    Profile('design_efield_monitor'),
]
adjoint_monitors = [
    Profile('design_efield_monitor'),
]

fom = BayerFilterFoM(
    polarization='TE',
    fwd_srcs=forward_sources,
    adj_srcs=adjoint_sources,
    fwd_monitors=forward_monitors,
    adj_monitors=adjoint_monitors,
    pos_max_freqs=list(range(60)),  # We're maximizing all 60 frequencies 
    neg_min_freqs=[],
)

Each FoM is also able to create its forward and adjoint simulations given a provided
"base simulation" to start from. It achieves this by disabling all sources except for
those specified, and by replacing its `Monitor`'s with their counterparts in the 
simulation.

So long as the FoM's monitors and sources share a name with a corresponding object in 
the base simulation, the `create_fwd_sim` and `create_adj_sim` methods will create 
their respective simulation variants automatically.

In [ ]:
base_sim = LumericalSimulation('simulation_example.json')

fwd_sim = fom.create_forward_sim(base_sim)[0]  # Access first element, as method returns a list
adj_sim = fom.create_adjoint_sim(base_sim)[0]

# Run the simulations and save the monitor data
fdtd = LumericalFDTD()

fdtd.save('fwd_sim.fsp', fwd_sim)
fdtd.save('adj_sim.fsp', adj_sim)

fdtd.addjob('fwd_sim.fsp')
fdtd.addjob('adj_sim.fsp')

fdtd.runjobs()
fdtd.reformat_monitor_data([fwd_sim, adj_sim])

Now that the FoM has been created, and its forward and adjoint simulations have been 
run, we can finally compute its functions.

In [ ]:
f = fom.compute_fom()

g = fom.compute_grad()

print(f'The FoM is {f}')
print(f'The gradient is {g}')

### Arithmetic Functions on `FoM`'s

### Spectral Weighting and Performance Weighting

## Defining a New Figure of Merit

The `FoM` class at its core is defined by two functions:
* A function that computes the FoM (`fom_func`)
* A function that computes the gradient of the FoM for optimization (`grad_func`)

That's it! In fact, to implement your own FoM class, all you need to do is provide these
two functions and it will be compatible with all other optimization code.

In the following examples, we use the following FoM:
$$FoM(x) = x^2 \qquad FoM'(x) = 2 x$$

In [ ]:
def fom_func(x):
    return x ** 2

def gradient_func(x):
    return 2 * x